In [14]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


BoW(Bag-of-Word)

텍스트를 수치 특성 벡터로 표현

1. 전체 문서에 대해 고유한 토큰(Token). 예를 들어, 단어로 이루어진 어휘 사전을 만든다.

2. 특정 문서에 각 단어가 얼마나 자주 등장하는지 헤아려 문서의 특성 벡터를 만든다.

In [15]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer() #BoW 만드는 Class..
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

In [18]:
print(count.vocabulary_)
#label 할당

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


이전 결과에서 볼 수 있듯이 어휘 사전은 고유 단어와 정수 인덱스가 매핑된 파이썬 딕셔너리에 저장되어 있습니다. 그다음 만들어진 특성 벡터를 출력해 봅시다:

특성 벡터의 각 인덱스는 CountVectorizer의 어휘 사전 딕셔너리에 저장된 정수 값에 해당됩니다. 예를 들어 인덱스 0에 있는 첫 번째 특성은 ‘and’ 단어의 카운트를 의미합니다. 이 단어는 마지막 문서에만 나타나네요. 인덱스 1에 있는 (특성 벡터의 두 번째 열) 단어 ‘is’는 세 문장에 모두 등장합니다. 특성 벡터의 이런 값들을 단어 빈도(term frequency) 라고도 부릅니다. 문서 d에 등장한 단어 t의 횟수를 tf (t,d)와 같이 씁니다.

In [17]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [19]:
np.set_printoptions(precision=2)

텍스트 데이터를 분석할 때 클래스 레이블이 다른 문서에 같은 단어들이 나타나는 경우를 종종 보게 됩니다. 일반적으로 자주 등장하는 단어는 유용하거나 판별에 필요한 정보를 가지고 있지 않습니다. 이 절에서 특성 벡터에서 자주 등장하는 단어의 가중치를 낮추는 기법인 tf-idf(term frequency-inverse document frequency)에 대해 배우겠습니다. tf-idf는 단어 빈도와 역문서 빈도(inverse document frequency)의 곱으로 정의됩니다:

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mtext>tf-idf</mtext>
  <mo stretchy="false">(</mo>
  <mi>t</mi>
  <mo>,</mo>
  <mi>d</mi>
  <mo stretchy="false">)</mo>
  <mo>=</mo>
  <mtext>tf (t,d)</mtext>
  <mo>&#xD7;</mo>
  <mtext>idf</mtext>
  <mo stretchy="false">(</mo>
  <mi>t</mi>
  <mo>,</mo>
  <mi>d</mi>
  <mo stretchy="false">)</mo>
</math>

여기에서 tf(t, d)는 이전 절에서 보았던 단어 빈도입니다. idf(t, d)는 역문서 빈도로 다음과 같이 계산합니다:

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mtext>idf</mtext>
  <mo stretchy="false">(</mo>
  <mi>t</mi>
  <mo>,</mo>
  <mi>d</mi>
  <mo stretchy="false">)</mo>
  <mo>=</mo>
  <mtext>log</mtext>
  <mfrac>
    <msub>
      <mi>n</mi>
      <mi>d</mi>
    </msub>
    <mrow>
      <mn>1</mn>
      <mo>+</mo>
      <mtext>df</mtext>
      <mo stretchy="false">(</mo>
      <mi>d</mi>
      <mo>,</mo>
      <mi>t</mi>
      <mo stretchy="false">)</mo>
    </mrow>
  </mfrac>
  <mo>,</mo>
</math>

\text{idf}(t,d) = \text{log}\frac{n_d}{1+\text{df}(d, t)},

n_d / 1+df(d,t) 이다 (1은 분모의 0을 피하기 위해)

여기에서는 전체 문서 개수이고 df(d, t)는 단어 t가 포함된 문서 d의 개수입니다. 분모에 상수 1을 추가하는 것은 선택 사항입니다. 훈련 샘플에 한 번도 등장하지 않는 단어가 있는 경우 분모가 0이 되지 않게 만듭니다. log는 문서 빈도 df(d, t)가 낮을 때 역문서 빈도 값이 너무 커지지 않도록 만듭니다.

사이킷런 라이브러리에는 CountVectorizer 클래스에서 만든 단어 빈도를 입력받아 tf-idf로 변환하는 TfidfTransformer 클래스가 구현되어 있습니다:

In [20]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2', 
                         smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs))
      .toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


이전 절에서 보았듯이 세 번째 문서에서 단어 ‘is’가 가장 많이 나타났기 때문에 단어 빈도가 가장 컸습니다. 동일한 특성 벡터를 tf-idf로 변환하면 단어 ‘is’는 비교적 작은 tf-idf를 가집니다(0.45). 이 단어는 첫 번째와 두 번째 문서에도 나타나므로 판별에 유용한 정보를 가지고 있지 않을 것입니다.

수동으로 특성 벡터에 있는 각 단어의 tf-idf를 계산해 보면 TfidfTransformer가 앞서 정의한 표준 tf-idf 공식과 조금 다르게 계산한다는 것을 알 수 있습니다. 사이킷런에 구현된 역문서 빈도 공식은 다음과 같습니다.

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mtext>idf</mtext>
  <mo stretchy="false">(</mo>
  <mi>t</mi>
  <mo>,</mo>
  <mi>d</mi>
  <mo stretchy="false">)</mo>
  <mo>=</mo>
  <mi>l</mi>
  <mi>o</mi>
  <mi>g</mi>
  <mfrac>
    <mrow>
      <mn>1</mn>
      <mo>+</mo>
      <msub>
        <mi>n</mi>
        <mi>d</mi>
      </msub>
    </mrow>
    <mrow>
      <mn>1</mn>
      <mo>+</mo>
      <mtext>df</mtext>
      <mo stretchy="false">(</mo>
      <mi>d</mi>
      <mo>,</mo>
      <mi>t</mi>
      <mo stretchy="false">)</mo>
    </mrow>
  </mfrac>
</math>

\text{idf} (t,d) = log\frac{1 + n_d}{1 + \text{df}(d, t)}

비슷하게 사이킷런에서 계산하는 tf-idf는 앞서 정의한 공식과 조금 다릅니다:



<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mtext>tf-idf</mtext>
  <mo stretchy="false">(</mo>
  <mi>t</mi>
  <mo>,</mo>
  <mi>d</mi>
  <mo stretchy="false">)</mo>
  <mo>=</mo>
  <mtext>tf</mtext>
  <mo stretchy="false">(</mo>
  <mi>t</mi>
  <mo>,</mo>
  <mi>d</mi>
  <mo stretchy="false">)</mo>
  <mo>&#xD7;</mo>
  <mo stretchy="false">(</mo>
  <mtext>idf</mtext>
  <mo stretchy="false">(</mo>
  <mi>t</mi>
  <mo>,</mo>
  <mi>d</mi>
  <mo stretchy="false">)</mo>
  <mo>+</mo>
  <mn>1</mn>
  <mo stretchy="false">)</mo>
</math>

\text{tf-idf}(t,d) = \text{tf}(t,d) \times (\text{idf}(t,d)+1)

일반적으로 tf-idf를 계산하기 전에 단어 빈도(tf)를 정규화하지만 TfidfTransformer 클래스는 tf-idf를 직접 정규화합니다. 사이킷런의 TfidfTransformer는 기본적으로 L2 정규화를 적용합니다(norm=’l2’). 정규화되지 않은 특성 벡터 v를 L2-노름으로 나누면 길이가 1인 벡터가 반환됩니다:

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <msub>
    <mi>v</mi>
    <mrow data-mjx-texclass="ORD">
      <mtext>norm</mtext>
    </mrow>
  </msub>
  <mo>=</mo>
  <mfrac>
    <mi>v</mi>
    <mrow>
      <mo data-mjx-texclass="ORD" stretchy="false">|</mo>
      <mo data-mjx-texclass="ORD" stretchy="false">|</mo>
      <mi>v</mi>
      <mo data-mjx-texclass="ORD" stretchy="false">|</mo>
      <msub>
        <mo data-mjx-texclass="ORD" stretchy="false">|</mo>
        <mn>2</mn>
      </msub>
    </mrow>
  </mfrac>
  <mo>=</mo>
  <mfrac>
    <mi>v</mi>
    <msqrt>
      <msubsup>
        <mi>v</mi>
        <mrow data-mjx-texclass="ORD">
          <mn>1</mn>
        </mrow>
        <mrow data-mjx-texclass="ORD">
          <mn>2</mn>
        </mrow>
      </msubsup>
      <mo>+</mo>
      <msubsup>
        <mi>v</mi>
        <mrow data-mjx-texclass="ORD">
          <mn>2</mn>
        </mrow>
        <mrow data-mjx-texclass="ORD">
          <mn>2</mn>
        </mrow>
      </msubsup>
      <mo>+</mo>
      <mo>&#x22EF;</mo>
      <mo>+</mo>
      <msubsup>
        <mi>v</mi>
        <mrow data-mjx-texclass="ORD">
          <mi>n</mi>
        </mrow>
        <mrow data-mjx-texclass="ORD">
          <mn>2</mn>
        </mrow>
      </msubsup>
    </msqrt>
  </mfrac>
  <mo>=</mo>
  <mfrac>
    <mi>v</mi>
    <mrow>
      <mrow data-mjx-texclass="ORD">
        <mo minsize="1.2em" maxsize="1.2em">(</mo>
      </mrow>
      <munderover>
        <mo data-mjx-texclass="OP">&#x2211;</mo>
        <mrow data-mjx-texclass="ORD">
          <mi>i</mi>
          <mo>=</mo>
          <mn>1</mn>
        </mrow>
        <mrow data-mjx-texclass="ORD">
          <mi>n</mi>
        </mrow>
      </munderover>
      <msubsup>
        <mi>v</mi>
        <mrow data-mjx-texclass="ORD">
          <mi>i</mi>
        </mrow>
        <mrow data-mjx-texclass="ORD">
          <mn>2</mn>
        </mrow>
      </msubsup>
      <msup>
        <mrow data-mjx-texclass="ORD">
          <mo minsize="1.2em" maxsize="1.2em">)</mo>
        </mrow>
        <mfrac>
          <mn>1</mn>
          <mn>2</mn>
        </mfrac>
      </msup>
    </mrow>
  </mfrac>
</math>

v_{\text{norm}} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v_{1}^{2} + v_{2}^{2} + \dots + v_{n}^{2}}} = \frac{v}{\big (\sum_{i=1}^{n} v_{i}^{2}\big)^\frac{1}{2}}

TfidfTransformer의 작동 원리를 이해하기 위해 세 번째 문서에 있는 단어 ‘is'의 tf-idf를 예로 들어 계산해 보죠.

세 번째 문서에서 단어 ‘is’의 단어 빈도는 3입니다(tf=3). 이 단어는 세 개 문서에 모두 나타나기 때문에 문서 빈도가 3입니다(df=3). 이에, 역문서 빈도는 0이 됩니다.

이제 tf-idf를 계산하기 위해 역문서 빈도에 1을 더하고 단어 빈도를 곱하면 3이 나온다

In [21]:
tf_is = 3
n_docs = 3
idf_is = np.log((n_docs+1) / (3+1))
tfidf_is = tf_is * (idf_is + 1)
print('tf-idf of term "is" = %.2f' % tfidf_is)

tf-idf of term "is" = 3.00


세 번째 문서에 있는 모든 단어에 대해 이런 계산을 반복하면 tf-idf 벡터 [3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0, 1.69, 1.29]를 얻습니다. 이 특성 벡터의 값은 앞서 사용했던 TfidfTransformer에서 얻은 값과 다릅니다. tf-idf 계산에서 빠트린 마지막 단계는 다음과 같은 L2-정규화입니다::

https://github.com/rickiepark/python-machine-learning-book-3rd-edition/blob/master/ch08/ch08.ipynb


참고해서 식 참조

In [22]:
tfidf = TfidfTransformer(use_idf=True, norm=None, smooth_idf=True)
raw_tfidf = tfidf.fit_transform(count.fit_transform(docs)).toarray()[-1]
raw_tfidf 

array([3.39, 3.  , 3.39, 1.29, 1.29, 1.29, 2.  , 1.69, 1.29])

In [23]:
l2_tfidf = raw_tfidf / np.sqrt(np.sum(raw_tfidf**2))
l2_tfidf

array([0.5 , 0.45, 0.5 , 0.19, 0.19, 0.19, 0.3 , 0.25, 0.19])

HTML 마크업 삭제처리, 구두점 기호 삭제처리 => 임시로 저장한 이모티콘을 처리 완료된 문자열 끝에 추가하여 구분

In [24]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [25]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

In [26]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [27]:

preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [28]:
df['review'] = df['review'].apply(preprocessor)

In [29]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer() #포터 어간 추출 알고리즘

def tokenizer(text):
    return text.split()


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [30]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [31]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [32]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

불용어(너무 자주 사용해서 사실상 분석에 있어서 의미가 없는 단어) 제거

In [34]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] 
if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [35]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer #tf-idf 두 기능 합체
from sklearn.model_selection import GridSearchCV #최적의 조합

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],#각 단어를 하나의 토큰으로 볼 지, 2~3개를 하나의 토큰으로 볼 지...
               'vect__stop_words': [stop, None], #불용어 사용여부
               'vect__tokenizer': [tokenizer, tokenizer_porter], #공백을 기준으로 하는 token으로 할지 포터 어간 추출 알고리즘 token으로 할지
               'clf__penalty': ['l1', 'l2'], #logistic regressor이니..
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False], #tf-idf에 idf X....
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0, solver='liblinear'))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           n_jobs=-1) #모든 cpu 코어 사용

너무 느릴 경우 훈련 세트를 줄이거나(성능 감소), 그리드에 지정한 매개변수(C) 삭제 등으로 코드 실행 시간 단축 가능

In [38]:
gs_lr_tfidf.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've...
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
 

In [39]:
print('최적의 매개변수 조합: %s ' % gs_lr_tfidf.best_params_)
print('CV 정확도: %.3f' % gs_lr_tfidf.best_score_)

최적의 매개변수 조합: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x000002AA02BB93A8>} 
CV 정확도: 0.897


In [40]:
clf = gs_lr_tfidf.best_estimator_
print('테스트 정확도: %.3f' % clf.score(X_test, y_test))

테스트 정확도: 0.899


gs_lr_tfidf.best_score_는 k-폴드 교차 검증의 평균 점수입니다. 즉 5-폴드 교차 검증으로 GridSearchCV 객체를 훈련하면 best_score_ 속성은 최상의 모델에 대한 5-폴드 점수의 평균을 반환합니다. 예를 들면 다음과 같습니다:

In [41]:
from sklearn.linear_model import LogisticRegression
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

np.random.seed(0)
np.set_printoptions(precision=6)
y = [np.random.randint(3) for i in range(25)]
X = (y + np.random.randn(25)).reshape(-1, 1)

cv5_idx = list(StratifiedKFold(n_splits=5, shuffle=False).split(X, y))
    
lr = LogisticRegression(random_state=123, multi_class='ovr', solver='lbfgs')
cross_val_score(lr, X, y, cv=cv5_idx)

array([0.4, 0.2, 0.6, 0.2, 0.4])

위 코드를 실행하면 클래스 레이블에 해당하는 랜덤한 정수 데이터셋을 만듭니다. 그다음 5-폴드의 인덱스(cv5_idx)를 cross_val_score 함수에 전달하여 5개의 정확도 점수를 받습니다. 이 점수가 다섯 개의 테스트 폴드에 대한 정확도 값입니다.

그다음 GridSearchCV 객체를 사용해 동일한 5-폴드 인덱스(cv5_idx)를 전달해 보죠:

In [42]:
from sklearn.model_selection import GridSearchCV

lr = LogisticRegression(solver='lbfgs', multi_class='ovr', random_state=1)
gs = GridSearchCV(lr, {}, cv=cv5_idx, verbose=3).fit(X, y) 

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END ..................................., score=0.400 total time=   0.0s
[CV 2/5] END ..................................., score=0.200 total time=   0.0s
[CV 3/5] END ..................................., score=0.600 total time=   0.0s
[CV 4/5] END ..................................., score=0.200 total time=   0.0s
[CV 5/5] END ..................................., score=0.400 total time=   0.0s


교차 검증의 평균 점수가 저장되며 최상의 모델에 대한 5-fold 교차검증에 대한 평균 점수를 의미

In [43]:
gs.best_score_

0.36000000000000004

In [44]:
lr = LogisticRegression(solver='lbfgs', multi_class='ovr', random_state=1)
cross_val_score(lr, X, y, cv=cv5_idx).mean()

0.36000000000000004

In [45]:
import os
import gzip


if not os.path.isfile('movie_data.csv'):
    if not os.path.isfile('movie_data.csv.gz'):
        print('Please place a copy of the movie_data.csv.gz'
              'in this directory. You can obtain it by'
              'a) executing the code in the beginning of this'
              'notebook or b) by downloading it from GitHub:'
              'https://github.com/rickiepark/python-machine-learning-'
              'book-3rd-edition/blob/master/ch08/movie_data.csv.gz')
    else:
        with gzip.open('movie_data.csv.gz', 'rb') as in_f, \
                open('movie_data.csv', 'wb') as out_f:
            out_f.write(in_f.read())

In [46]:
import numpy as np
import re
from nltk.corpus import stopwords


# `stop` 객체를 앞에서 정의했지만 이전 코드를 실행하지 않고
# 편의상 여기에서부터 코드를 실행하기 위해 다시 만듭니다.
stop = stopwords.words('english')


def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

#문쇼ㅓ 하나씩 일겅서 반환하는 Stream docs

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # 헤더 넘기기
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

#호출마다 text와 label 변환(Sample)
#Return은 반환 즉시 함수 종료, Yield는 계속 돌아감

In [47]:
next(stream_docs(path='movie_data.csv'))
#함수 체크

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [52]:
def get_minibatch(doc_stream, size): #위의 Stream docs에서 문서를 읽어 size 매개변수에서 지정한 만큼 문서를 반환하는 get_minibatch 함수
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

CounterVectorize, TfidfVectorize 클래스는 모두 클래스 전체 어휘 사전을 메모리에 가지고 있어야 하는데, 온라인 학습(조금씩 데이터를 가져오는) 상황에서는 부적합하다.

HashingVectorizer 사용

In [67]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier


vect = HashingVectorizer(decode_error='ignore', #UTF 파일 디코딩 시 에러 처리 하는 방법을 정할 수 있다.
                         n_features=2**21, #Hasing의 budget(너무 작으면 Hashing 충돌 가능)
                         preprocessor=None, 
                         tokenizer=tokenizer)

In [68]:

from distutils.version import LooseVersion as Version

# 사이킷런 1.3버전을 사용하는 경우 'log'를 'log_loss'로 바꾸어 사용하세요.
clf = SGDClassifier(loss='log', random_state=1) #logistics 회귀 문제를 푸는 경사하강법

doc_stream = stream_docs(path='movie_data.csv')

In [69]:
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


In [70]:
X_test, y_test = get_minibatch(doc_stream, size=5000) #'5000'개로 모델 성능 평가
X_test = vect.transform(X_test)
print('정확도: %.3f' % clf.score(X_test, y_test))

정확도: 0.868


In [71]:
clf = clf.partial_fit(X_test, y_test)

LDA(잠재 디리클레 할당, Latent Dirichlet Allocation, LDA)

레이블이 없는 텍스트 문서에 토픽을 할당하여 모델링 할 때 좋다. 데이터를 feature로 사용하여 분석한다

In [72]:

import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [73]:

from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df=.1,
                        max_features=5000)
X = count.fit_transform(df['review'].values) #review -> counterVectorize -> features

In [74]:
from sklearn.decomposition import LatentDirichletAllocation


lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch')
X_topics = lda.fit_transform(X)

In [75]:
n_top_words = 5 #가장 중요한 단어 5개 출력
feature_names = count.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i]
                    for i in topic.argsort()\
                        [:-n_top_words - 1:-1]])) #오른 차순이기 때문에 역순으로

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children girl
Topic 3:
american war dvd music tv
Topic 4:
human audience cinema art sense
Topic 5:
police guy car dead murder
Topic 6:
horror house sex girl woman
Topic 7:
role performance comedy actor performances
Topic 8:
series episode war episodes tv
Topic 9:
book version original read novel
Topic 10:
action fight guy guys cool


각 토픽에서 가장 중요한 단어 다섯 개를 기반으로 LDA가 다음 토픽을 구별했다고 추측할 수 있습니다.

1. 대체적으로 형편없는 영화(실제 토픽 카테고리가 되지 못함)
2. 가족 영화
3. 전쟁 영화
4. 예술 영화
5. 범죄 영화
6. 공포 영화
7. 코미디 영화
8. TV 쇼와 관련된 영화
9. 소설을 원작으로 한 영화
10. 액션 영화
카테고리가 잘 선택됐는지 확인하기 위해 공포 영화 카테고리에서 3개 영화의 리뷰를 출력해 보죠(공포 영화는 카테고리 6이므로 인덱스는 5입니다):

In [76]:
horror = X_topics[:, 5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\n공포 영화 #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')


공포 영화 #1:
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

공포 영화 #2:
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

공포 영화 #3:
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...


IMDb 영화 리뷰 데이터셋과 비슷한 네이버 영화 리뷰 데이터셋(https://github.com/e9t/nsmc)을 사용해 한글 문장의 감성 분류 예제를 다루어 보겠습니다. 이 데이터는 네이버 영화 사이트에 있는 리뷰 20만 개를 모은 것입니다. 네이버 영화 리뷰 데이터셋 깃허브에서 직접 데이터를 다운로드 받아도 되지만 편의를 위해 이 책의 깃허브의 ch09 폴더에 데이터셋을 넣어 놓았습니다.

20만개 데이터 중 15만개는 훈련 데이터셋으로 ratings_train.txt 파일에 저장되어 있고 5만개는 테스트 데이터셋으로 ratings_test.txt 파일에 저장되어 있습니다. 리뷰의 길이는 140을 넘지 않습니다. 부정 리뷰는 1~4까지 점수를 매긴 리뷰이고 긍정 리뷰는 6~10까지 점수를 매긴 리뷰입니다. 훈련 데이터셋과 테스트 데이터셋의 부정과 긍정 리뷰는 약 50%씩 구성되어 있습니다.

한글은 영어와 달리 조사와 어미가 발달해 있기 때문에 BoW나 어간 추출보다 표제어 추출 방식이 적합합니다. 이런 작업을 형태소 분석이라 부릅니다. 파이썬에서 한글 형태소 분석을 하기 위한 대표적인 패키지는 konlpy와 soynlp입니다. 두 패키지를 모두 사용해 네이버 영화 리뷰를 긍정과 부정으로 분류해 보겠습니다.

먼저 이 예제를 실행하려면 konlpy와 soynlp가 필요합니다.

In [1]:
import konlpy
import pandas as pd
import numpy as np

감성 분류를 시작하기 전에 훈련 데이터셋과 테스트 데이터셋을 각각 판다스 데이터프레임으로 읽은 후 넘파이 배열로 준비하겠습니다. 먼저 훈련 데이터셋부터 읽어 보죠. ratings_train.txt 파일은 하나의 리뷰가 한 행을 구성하며 각 필드는 탭으로 구분되어 있기 때문에 판다스의 read_csv() 함수로 간편하게 읽어 들일 수 있습니다. read_csv()는 기본적으로 콤마를 기준으로 필드를 구분하므로 delimiter='\t'으로 지정하여 탭으로 변경합니다. 기본적으로 판다스는 빈 문자열을 NaN으로 인식합니다. 빈 문자열을 그대로 유지하기 위해 keep_default_na 매개변수를 False로 지정합니다.

In [2]:
df_train = pd.read_csv('ratings_train.txt', 
                       delimiter='\t', keep_default_na=False)

In [3]:
df_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


데이터프레임의 열을 선택하여 Series 객체의 values 속성을 사용하면 document 열과 label 열을 넘파이 배열로 저장할 수 있습니다. 각각 훈련 데이터셋의 특성과 타깃 값으로 저장합니다.

In [4]:
X_train = df_train['document'].values
y_train = df_train['label'].values

In [5]:
df_test = pd.read_csv('ratings_test.txt', 
                      delimiter='\t', keep_default_na=False)

X_test = df_test['document'].values
y_test = df_test['label'].values

In [6]:
print(len(X_train), np.bincount(y_train))
print(len(X_test), np.bincount(y_test))

150000 [75173 74827]
50000 [24827 25173]


훈련 데이터셋과 테스트 데이터셋을 준비했으므로 형태소 분석기를 사용해 본격적인 감성 분류 작업을 시작해 보겠습니다.

konlpy는 5개의 한국어 형태소 분석기를 파이썬 클래스로 감싸서 제공하는 래퍼 패키지입니다. konlpy가 제공하는 형태소 분석기에 대한 자세한 내용은 온라인 문서(https://konlpy.org/ko/latest/)를 참고하세요. 이 예에서는 스칼라로 개발된 open-korean-text 한국어 처리기(https://github.com/open-korean-text/open-korean-text)를 제공하는 Okt 클래스를 사용해 보겠습니다. open-korean-text는 비교적 성능이 높고 별다른 설치 없이 구글 코랩에서도 바로 사용할 수 있습니다.

konlpy.tag 패키지에서 Okt 클래스를 임포트하고 객체를 만든 다음 훈련 데이터셋에 있는 문장 하나를 morphs() 메서드로 형태소로 나누어 보겠습니다.

In [7]:
from konlpy.tag import Okt

okt = Okt()
print(X_train[4])
print(okt.morphs(X_train[4]))

사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
['사이', '몬페', '그', '의', '익살스런', '연기', '가', '돋보였던', '영화', '!', '스파이더맨', '에서', '늙어', '보이기만', '했던', '커스틴', '던스트', '가', '너무나도', '이뻐', '보였다']


한글 문장에서 조사와 어미가 잘 구분되어 출력된 것을 볼 수 있습니다. '사이몬페그'와 같은 고유 명사는 처리하는데 어려움을 겪고 있네요. 완벽하지는 않지만 이 클래스를 사용해 분류 문제를 풀어 보겠습니다.

TfidfVectorzier는 기본적으로 공백을 기준으로 토큰을 구분하지만 tokenizer 매개변수에 토큰화를 위한 사용자 정의 함수를 전달할 수 있습니다. 따라서 앞서 테스트했던 okt.morphs 메서드를 전달하면 형태소 분석을 통해 토큰화를 수행할 수 있습니다. tokenizer 매개변수를 사용할 때 패턴token_pattern=None으로 지정하여 token_pattern 매개변수가 사용되지 않는다는 경고 메시지가 나오지 않게 합니다.

TfidfVectorzier을 ngram_range=(1, 2)로 설정하여 유니그램과 바이그램을 사용하고 min_df=3으로 지정하여 3회 미만으로 등장하는 토큰은 무시합니다. 또한 max_df=0.9로 두어 가장 많이 등장하는 상위 10%의 토큰도 무시하겠습니다. 이런 작업이 불용어로 생각할 수 있는 토큰을 제거할 것입니다.

컴퓨팅 파워가 충분하다면 하이퍼파라미터 탐색 단계에서 TfidfVectorzier의 매개변수도 탐색해 보는 것이 좋습니다. 여기에서는 임의의 매개변수 값을 지정하여 데이터를 미리 변환하고 하이퍼파라미터 탐색에서는 분류기의 매개변수만 탐색하겠습니다.

노트: 토큰 데이터를 생성하는 시간이 많이 걸리므로 주피터 노트북에서는 다음 번 실행 때 이 과정을 건너 뛸 수 있도록 이 데이터를 한 번 생성하여 npz 파일로 저장합니다.

In [8]:
import os
from scipy.sparse import save_npz, load_npz
from sklearn.feature_extraction.text import TfidfVectorizer

if not os.path.isfile('okt_train.npz'):
    tfidf = TfidfVectorizer(ngram_range=(1, 2), 
                            min_df=3,
                            max_df=0.9,
                            tokenizer=okt.morphs, 
                            token_pattern=None)
    tfidf.fit(X_train)
    X_train_okt = tfidf.transform(X_train)
    X_test_okt = tfidf.transform(X_test)
    save_npz('okt_train.npz', X_train_okt)
    save_npz('okt_test.npz', X_test_okt)
else:
    X_train_okt = load_npz('okt_train.npz')
    X_test_okt = load_npz('okt_test.npz')

X_train_okt와 X_test_okt가 준비되었으므로 SGDClassifier 클래스를 사용해 감성 분류 문제를 풀어 보겠습니다. 탐색할 SGDClassifier의 매개변수는 규제를 위한 alpha 매개변수입니다. RandomizedSearchCV 클래스를 사용하기 위해 loguniform 함수로 탐색 범위를 지정하겠습니다. 여기에서는 SGDClassifier의 손실 함수로 로지스틱 손실('log')을 사용하지만 다른 손실 함수를 매개변수 탐색에 포함할 수 있습니다. 총 반복 회수(n_iter)는 50회로 지정합니다. 만약 CPU 코어가 여러개라면 n_jobs 매개변수를 1 이상으로 설정하여 수행 속도를 높일 수 있습니다.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.utils.fixes import loguniform

# 사이킷런 1.3버전을 사용하는 경우 'log'를 'log_loss'로 바꾸어 사용하세요.
sgd = SGDClassifier(loss='log', random_state=1)
param_dist = {'alpha': loguniform(0.0001, 100.0)}

rsv_okt = RandomizedSearchCV(estimator=sgd,
                             param_distributions=param_dist,
                             n_iter=50,
                             random_state=1,
                             verbose=1)
rsv_okt.fit(X_train_okt, y_train)

In [ ]:
print(rsv_okt.best_score_)
print(rsv_okt.best_params_)

In [ ]:
rsv_okt.score(X_test_okt, y_test)

약 82%의 정확도를 냈습니다. 간단한 작업으로 꽤 좋은 성능을 냈습니다. konlpy의 다른 형태소 분석 클래스를 사용하거나 SGDClassifier 외에 다른 분류기를 시도하지 않을 이유는 없습니다. 충분한 컴퓨팅 파워가 없다면 사이킷런 0.24버전에서 추가되는 HalvingRandomSearchCV 클래스를 사용해 볼 수도 있습니다.

이번에는 또 다른 파이썬 형태소 분석기인 soynlp를 사용해 보겠습니다. soynlp는 순수하게 파이썬으로 구현된 형태소 분석 패키지입니다. 깃허브(https://github.com/lovit/soynlp)에는 소스 코드 뿐만 아니라 다양한 튜토리얼도 함께 제공합니다. soynlp는 3개의 토큰화 클래스를 제공합니다. 기본적으로 띄어쓰기가 잘 되어 있다면 LTokenizer가 잘 맞습니다. 이외에는 MaxScoreTokenizer와 RegexTokenizer가 있습니다. 이 예에서는 LTokenizer를 사용해 보겠습니다. 먼저 soynlp.tokenizer에서 LTokenizer를 임포트합니다.

In [ ]:
from soynlp.tokenizer import LTokenizer

soynlp는 말뭉치의 통계 데이터를 기반으로 동작하기 때문에 기본 LTokenizer 객체로는 공백으로만 토큰화를 수행합니다. LTokenizer에 필요한 통계 데이터를 생성하기 위해서 WordExtractor를 사용해 보겠습니다.

In [ ]:
from soynlp.word import WordExtractor

WordExtractor 객체를 만든 후 train() 메서드에 X_train을 전달하여 훈련합니다. 훈련이 끝나면 word_scores() 메서드에서 단어의 점수를 얻을 수 있습니다. 반환된 scores 객체는 단어마다 결합 점수(cohesion score)와 브랜칭 엔트로피(branching entropy)를 가진 딕셔너리입니다.

In [ ]:
word_ext = WordExtractor()
word_ext.train(X_train)
scores = word_ext.word_scores()

soynlp 깃허브의 튜토리얼(https://github.com/lovit/soynlp/blob/master/tutorials/wordextractor_lecture.ipynb)을 따라 결합 점수(cohesion_forward)와 브랜칭 엔트로피(right_branching_entropy)에 지수를 취한 값에 곱하여 최종 점수를 만들겠습니다.

In [ ]:
import math

score_dict = {key: scores[key].cohesion_forward *
              math.exp(scores[key].right_branching_entropy) 
              for key in scores}


이제 이 점수를 LTokenizer의 scores 매개변수로 전달하여 객체를 만들고 앞에서 테스트한 샘플에 다시 적용해 보겠습니다.

In [ ]:
lto = LTokenizer(scores=score_dict)

print(lto.tokenize(X_train[4]))

In [ ]:
if not os.path.isfile('soy_train.npz'):
    tfidf = TfidfVectorizer(ngram_range=(1, 2),
                            min_df=3,
                            max_df=0.9,
                            tokenizer=lto.tokenize, 
                            token_pattern=None)
    tfidf.fit(X_train)
    X_train_soy = tfidf.transform(X_train)
    X_test_soy = tfidf.transform(X_test)
    save_npz('soy_train.npz', X_train_soy)
    save_npz('soy_test.npz', X_test_soy)
else:
    X_train_soy = load_npz('soy_train.npz')
    X_test_soy = load_npz('soy_test.npz')

동일한 SGDClassifier 객체와 매개변수 분포를 지정하고 하이퍼파라미터 탐색을 수행해 보겠습니다.

In [ ]:
rsv_soy = RandomizedSearchCV(estimator=sgd,
                             param_distributions=param_dist,
                             n_iter=50,
                             random_state=1,
                             verbose=1)
rsv_soy.fit(X_train_soy, y_train)

soynlp를 사용했을 때 최상의 점수와 매개변수는 다음과 같습니다.

In [ ]:
print(rsv_soy.best_score_)
print(rsv_soy.best_params_)

In [ ]:
#마지막 테스트 셋 확인
rsv_soy.score(X_test_soy, y_test)

Okt를 사용했을 때 보다는 조금 더 낮지만 약 81% 이상의 정확도를 얻었습니다.